In [1]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-11-27_varle_lt.ipynb

import pandas as pd
import time
import sqlite3

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of import properties_of
from my_lib.my_db_functions import save_df_to_sqlite

print('Bibliotekos importuotos.')

Bibliotekos importuotos.


In [2]:
print('WebDriver started.',end=' ')

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dronai/'
driver.get(url)
time.sleep(3)
source = driver.page_source
# WebDriver stop!
driver.close()

print(f'Prskaityta {url}. WebDriver closed.')
# 14.9s

WebDriver started. Prskaityta https://www.varle.lt/dronai/. WebDriver closed.


In [3]:
bs = BeautifulSoup(source,'html.parser')

pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())

print(f'Total pages: {total_pages}')

items_df = pd.DataFrame(columns=('id','price','url')) # šios programos rezultatas

Total pages: 4


In [4]:
items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    items_df.loc[len(items_df)] = {'id':item_id,'price':item_price,'url':item_url}

print(len(items_df))

38


In [5]:

# WebDriver start!
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)

for page_number in range(2,total_pages+1):

    url = f'https://www.varle.lt/dronai/?p={page_number}'
    driver.get(url)
    time.sleep(3)
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        items_df.loc[len(items_df)] = {'id':item_id,'price':item_price,'url':item_url}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='first',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

# WebDriver stop!
driver.close()


Prskaityta https://www.varle.lt/dronai/?p=2
Get 76 items. Unique 76 items.
Prskaityta https://www.varle.lt/dronai/?p=3
Get 114 items. Unique 114 items.
Prskaityta https://www.varle.lt/dronai/?p=4
Get 117 items. Unique 117 items.


In [6]:
# Abu skaičiai sutampa, jei dublikatai pašalinti.
grupuotas_df=items_df.groupby('id')
print(len(grupuotas_df),len(items_df))

117 117


In [ ]:
# Save to SQLite3
save_df_to_sqlite(items_df,"./my_scrap_working_data.db")

Lentelė 'items_df' sėkmingai įrašyta į DB.
